In [2]:
# Labeling the data
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import Image

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import normalize, to_categorical

2023-04-03 03:27:39.264875: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-03 03:27:39.310126: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-03 03:27:39.311071: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 03:27:40.186745: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
image_directory='Data'
Fake=os.listdir(image_directory+ '/Fake')
Real=os.listdir(image_directory+ '/Real')

print('Fake Images: ', len(Fake))
print('Real Images: ',len(Real))

Fake Images:  101
Real Images:  1000
